In [12]:
import os
import torch
import pandas as pd
import gradio as gr
from transformers import (
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast,
)
from functools import lru_cache

# ==============================
# 0) CONFIG
# ==============================
SENTIMENT_MODEL_PATH = "/content/drive/MyDrive/models/distilbert_model"
CATEGORIZED_DATA_PATH = "/content/drive/MyDrive/amazon_reviews_clustered_and_categorized.csv"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

@lru_cache(maxsize=1)
def load_df() -> pd.DataFrame:
    df = pd.read_csv(CATEGORIZED_DATA_PATH)
    if "asin" in df.columns:
        df["asin"] = df["asin"].astype(str)
    if "full_review" in df.columns:
        df["full_review"] = df["full_review"].astype(str).fillna("")
    return df

@lru_cache(maxsize=1)
def load_sentiment_model():
    tok = DistilBertTokenizerFast.from_pretrained(SENTIMENT_MODEL_PATH)
    model = DistilBertForSequenceClassification.from_pretrained(SENTIMENT_MODEL_PATH)
    model.to(DEVICE)
    model.eval()
    return tok, model

# ==============================
# Functions
# ==============================

def predict_sentiment_gradio(text: str, progress=gr.Progress(track_tqdm=False)) -> str:
    if not text.strip():
        return "Please enter some text."
    tok, model = load_sentiment_model()
    inputs = tok(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        pred = outputs.logits.argmax(-1).item()
    labels = ["negative", "neutral", "positive"]
    return labels[pred] if pred < len(labels) else str(pred)

def display_category_distribution() -> str:
    df = load_df()
    counts = df["meta_category"].value_counts().reset_index()
    counts.columns = ["Meta-Category", "Number of Reviews"]
    return counts.to_html(index=False)

def download_category_counts() -> str:
    df = load_df()
    counts = df["meta_category"].value_counts().reset_index()
    counts.columns = ["Meta-Category", "Number of Reviews"]
    path = "/content/category_counts.csv"
    counts.to_csv(path, index=False)
    return path

# ==============================
# Build UI
# ==============================

def build_ui():
    df = load_df()
    with gr.Blocks(theme=gr.themes.Soft(), title="Amazon Review Analysis") as demo:
        gr.Markdown("# Amazon Review Sentiment Analyser")
        with gr.Row():
            with gr.Column():
                gr.Markdown("### Sentiment Analysis")
                text = gr.Textbox(lines=4, label="Enter Review")
                out = gr.Textbox(label="Predicted Sentiment")
                gr.Button("Analyze").click(predict_sentiment_gradio, text, out)
            with gr.Column():
                gr.Markdown("### Category Overview")
                table = gr.HTML(display_category_distribution())
                refresh = gr.Button("Refresh")
                csv_btn = gr.Button("Download CSV")
                csv_file = gr.File()
                refresh.click(lambda: display_category_distribution(), None, table)
                csv_btn.click(download_category_counts, None, csv_file)
    return demo

if __name__ == "__main__":
    try:
        from google.colab import drive
        drive.mount("/content/drive")
    except Exception:
        pass
    app = build_ui()
    app.queue().launch(debug=True, share=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4e2dff9b4cd43388e7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8856dbd90cfbb399a3.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://423750a8ff478ba34e.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://4e2dff9b4cd43388e7.gradio.live
